In [1]:
import os, sys, re, time, math
import numpy as np
import pandas as pd
from radiomics import featureextractor, getTestCase
import radiomics
import SimpleITK as sitk
import matplotlib.pyplot as plt
import shutil
import json
import contextlib
import time
import multiprocessing as mp
import psutil

%matplotlib notebook

In [2]:
def create_mask(raw, nii, out_name = 'Out.nii.gz', out_dir = None, *kargs):
    p = re.compile('\.nii$|\.nii\.gz$')
    m = p.search(out_name)

    if not m:
        out_name = out_name + '.nii.gz'
    if out_dir == None:
        out_dir = os.path.dirname(os.path.abspath(nii))
        
    full_out = os.path.join(out_dir, out_name)
    if os.path.isfile(full_out):
        return os.path.abspath(nii), full_out 
    
    if os.path.isfile(full_out):
        os.remove(full_out)
    
    reader = sitk.ImageFileReader()
    reader.SetImageIO("NiftiImageIO")
    reader.SetFileName(nii)
    image = reader.Execute();
    
    reader = sitk.ImageFileReader()
    reader.SetImageIO("NiftiImageIO")
    reader.SetFileName(raw)
    base_img = reader.Execute();
    
    arr = sitk.GetArrayFromImage(image)    
    bool_arr = np.zeros((arr.shape), dtype = bool)
    
    for val in kargs:
        if isinstance(val, list):
            for v in val:
                temp_arr = arr == v
                bool_arr = bool_arr + temp_arr
        else:
            temp_arr = arr == val
            bool_arr = bool_arr + temp_arr
    
    mask_arr = bool_arr.astype(float)
    temp_image = sitk.GetImageFromArray(mask_arr)
    temp_image.CopyInformation(base_img)
    sitk.WriteImage(temp_image, full_out)
    
    return os.path.abspath(nii), full_out



def merge_masks(out_name, *kargs):
    if os.path.isfile(out_name):
        os.remove(out_name)
    paths = []
    
    for arg in kargs:
        if isinstance(arg, list):
            for val in arg:
                paths.append(val)
        else:
            paths.append(paths)
    
    arr = None
    
    for path in paths:    
        reader = sitk.ImageFileReader()
        reader.SetImageIO("NiftiImageIO")
        reader.SetFileName(path)
        image = reader.Execute();
        img_arr = sitk.GetArrayFromImage(image)
        if arr is None:
           arr = np.zeros((img_arr.shape))
        arr = arr + img_arr
        
        
    final = (arr > 0).astype(float)
    temp_image = sitk.GetImageFromArray(final)
    temp_image.CopyInformation(image)
    sitk.WriteImage(temp_image, out_name) 
    
def make_multi_color_figure(out_name, path):
    if not os.path.isdir(path):
        return 
    paths = os.listdir(path)
    rem_ind = []
    for enum, p in enumerate(paths):
        if 'merged_mask' in p or 'multi_color' in p:
            rem_ind.append(enum)
    for i in reversed(rem_ind):
        del paths[i]
    for p in paths:
        print(p)
    return 
    paths = [os.path.join(out_name, p) for p in paths]
    merge_multi_masks(out_path, paths)


def merge_multi_masks(out_name, *kargs):
    if os.path.isfile(out_name):
        os.remove(out_name)
    paths = []
    
    for arg in kargs:
        if isinstance(arg, list):
            for val in arg:
                paths.append(val)
        else:
            paths.append(paths)
    
    arr = None
    
    for enum, path in enumerate(paths):    
        reader = sitk.ImageFileReader()
        reader.SetImageIO("NiftiImageIO")
        reader.SetFileName(path)
        image = reader.Execute();
        img_arr = sitk.GetArrayFromImage(image) * (enum+1)
        if arr is None:
           arr = np.zeros((img_arr.shape))
        arr = arr + img_arr
        
    temp_image = sitk.GetImageFromArray(arr)
    temp_image.CopyInformation(image)
    sitk.WriteImage(temp_image, out_name)
    
def create_structure_masks(subj_dir):
    dg_list = [51, 52, 50, 11, 49, 10, 53, 17, 12, 13, 26, 58, 54, 18, 255, 254, 253, 252, 251]
    bstem_list = [60, 16, 28]
    
    struc_dir = os.path.join(subj_dir, 'structure_masks')
    os.makedirs(struc_dir, exist_ok = True)
    files = os.listdir(subj_dir)
    t1, t2, seg = None, None, None
    
    for f in files:
        if 'T1w_hires.nii.gz' in f:
            t1 = os.path.join(subj_dir, f)
        if 'T2w_hires.nii.gz' in f:
            t2 = os.path.join(subj_dir, f)
        if 'aseg.hires.nii.gz' in f:
            seg = os.path.join(subj_dir, f)            
            
    _, right_cere_mask = create_mask(t1, seg, 'right_cereb_mask', struc_dir, 45,46,47) 
    _, left_cere_mask = create_mask(t1, seg, 'left_cereb_mask', struc_dir, 6,7,8)
    _, deep_grey = create_mask(t1, seg, 'deep_grey_mask', struc_dir, dg_list)
    _, right_wm = create_mask(t1, seg, 'right_wm_mask', struc_dir, 41)
    _, right_gm = create_mask(t1, seg, 'right_gm_mask', struc_dir, 42)
    _, left_wm = create_mask(t1, seg, 'left_wm_mask', struc_dir, 2)
    _, left_gm = create_mask(t1, seg, 'left_gm_mask', struc_dir, 3)
    _, brain_stem = create_mask(t1, seg, 'brain_stem_mask', struc_dir, bstem_list)
    
    mask_files = [right_cere_mask, left_cere_mask, deep_grey, right_wm, right_gm, left_wm, left_gm, brain_stem]
    name_list = [
        'Right_cerebellum', 
        'Left_cerebellum',
        'Deep_grey', 
        'Right_wm', 
        'Right_gm', 
        'Left_wm',
        'Left_gm',
        'Brain_stem']
    
    merged_file = os.path.join(struc_dir, 'merged_mask.nii.gz')
    if not os.path.isfile(merged_file):
        merge_masks(merged_file, mask_files)
    
    return mask_files, name_list


def extract_volume(path):
    image = sitk.ReadImage(path, imageIO="NiftiImageIO")
    x, y, z = image.GetSpacing()
    voxel_volume = x*y*z
    
    img_arr = sitk.GetArrayFromImage(image)
    
    return img_arr.sum() * voxel_volume, img_arr.sum()

def extract_volumes(out_dir, subj_id, path_list, name_list):
    volumes = {}
    out_file = os.path.join(out_dir, str(subj_id) + '_Volumes.json')
    
    if not os.path.isfile(out_file):
        for path, name in zip(path_list, name_list):
            vol, _ = extract_volume(path)
            volumes[name] = vol


        with open(out_file, 'w') as f:
            json.dump(volumes, f, indent = 4)

def run_radiomics(out_dir, subj_id, struc_file, path_list, name_list):
            extractor = featureextractor.RadiomicsFeatureExtractor()

            for path, name in zip(path_list, name_list):
                out_file = os.path.join(out_dir, str(subj_id) + '_' + name + '_Radiomics.json')
                if os.path.isfile(out_file):
                    continue
                result = extractor.execute(struc_file, path)
                clean_dic = {}
                for i in result.items():
                    if isinstance(i[1], np.ndarray):
                        clean_dic[i[0]] = float(i[1].tolist())
                    else:
                        clean_dic[i[0]] = i[1]
                
                with open(out_file, 'w') as f:
                    json.dump(clean_dic, f, indent = 4)

def check_files(*kargs):
    files = []
    files.extend(kargs)
    for count, f in enumerate(files):
        if isinstance(f, list):
            files.extend(f)
            del files[count]
    for f in files:
        if not os.path.isfile(f):
            return False
    return True

def run_all(path, t):
    subj = os.path.basename(path)
    result_folder = os.path.join(path, 'results')
    os.makedirs(result_folder, exist_ok = True)
    
    t1 = None
    for p in os.listdir(path):
        if 'T1w_hires.nii.gz' in p:
            t1 = os.path.join(path, p)

    mask_files, name_list = create_structure_masks(path)
    extract_volumes(result_folder, subj, mask_files, name_list) 
    if t1 is not None:
        run_radiomics(result_folder, subj, t1, mask_files, name_list)
        
def determine_threads(tmin, tmax, percent=15):
        usage = psutil.cpu_percent(interval=1, percpu=True)
        thread_avail = len([1 for i in usage if i < percent])
        
        return max(tmin, min(tmax, thread_avail))
        
def det_free_threads(free=5, percent=15):
    total = os.cpu_count()
    if free < 1:
        free = int(total * free)
    usage = psutil.cpu_percent(interval=1, percpu=True)
    thread_free = len([1 for i in usage if i < percent])
    if thread_free > free: 
        return thread_free - free, total - (thread_free - free)
    else:
        return 0, total       

In [4]:
storage_dir = '/home/billy/NAS/project/file_storage'
dirs = os.listdir(storage_dir)
dirs = [os.path.join(storage_dir, x) for x in dirs if os.path.isdir(os.path.join(storage_dir,x))]
dirs.sort(key = lambda x: int(os.path.basename(x)))

process = []
dir_copy = dirs.copy()
complete = False
count = 1
total = len(dirs)

try:
    start_time = time.time()
    while not complete:
        #thread_avail = determine_threads(4, 15, 15)
        free_threads, used_threads = det_free_threads(0.25, 15)
        #if len(process) < thread_avail:
        if free_threads > 0:
            arg = dir_copy.pop(0)
            subj = os.path.basename(arg)
            current_time = (time.time() - start_time)/60
            print("Running subject {}, {:04d}/{:04d}," 
                  "Total time: {:06.2f} mins, on {:02d}/{:02d} threads.".format(subj, 
                                                                        count, 
                                                                        total, 
                                                                        current_time, 
                                                                        len(process)+1,
                                                                        free_threads))
            count+=1
            p = mp.Process(target=run_all, args=(arg,start_time))
            p.start()
            process.append(p)

        for p in process:
            if not p.is_alive():
                process.remove(p)


        if len(dir_copy) == 0:
            for p in process:
                p.join()
            complete = True
            
except KeyboardInterrupt:
    print('\n\tEndng on subject {}'.format(subj))

except Exception as e:
    print('\n\tError on subject {}'.format(subj))
    print(e)

Running subject 100206, 0001/1112,Total time: 000.02 mins, on 01/14 threads.
Running subject 100307, 0002/1112,Total time: 000.03 mins, on 02/14 threads.
Running subject 100408, 0003/1112,Total time: 000.05 mins, on 02/14 threads.
Running subject 100610, 0004/1112,Total time: 000.07 mins, on 02/14 threads.
Running subject 101006, 0005/1112,Total time: 000.08 mins, on 02/14 threads.
Running subject 101107, 0006/1112,Total time: 000.10 mins, on 02/14 threads.
Running subject 101309, 0007/1112,Total time: 000.12 mins, on 02/14 threads.
Running subject 101410, 0008/1112,Total time: 000.13 mins, on 02/14 threads.
Running subject 101915, 0009/1112,Total time: 000.15 mins, on 02/14 threads.
Running subject 102008, 0010/1112,Total time: 000.17 mins, on 02/14 threads.
Running subject 102109, 0011/1112,Total time: 000.19 mins, on 02/14 threads.
Running subject 102311, 0012/1112,Total time: 000.20 mins, on 02/14 threads.
Running subject 102513, 0013/1112,Total time: 000.22 mins, on 02/14 threads.

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Running subject 118730, 0095/1112,Total time: 003.62 mins, on 15/01 threads.


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Running subject 118831, 0096/1112,Total time: 010.55 mins, on 15/01 threads.


...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Running subject 994273, 1111/1112,Total time: 1905.86 mins, on 16/01 threads.


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Running subject 995174, 1112/1112,Total time: 1906.41 mins, on 15/01 threads.


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

In [ ]:
color_loc = '/home/billy/NAS/project/file_storage/100307/structure_masks'
out_file = os.path.join(color_loc, 'multi_color.nii.gz')
make_multi_color_figure(out_file, color_loc)